# AymaraSDK + OpenAI Minimal Example

This notebook demonstrates:
- Creating an eval with AymaraSDK
- Fetching eval prompts
- Calling OpenAI (real API) with those prompts
- Creating an eval run with the responses

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-ai python-dotenv`

In [ ]:
# Environment and imports
import os

from dotenv import load_dotenv

load_dotenv()

import openai

from aymara_ai import AymaraAI
from aymara_ai.types import (
    EvalPrompt,
    EvalResponseParam,
)

## Set up API keys

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

## Instantiate the AymaraSDK client

In [ ]:
client = AymaraAI()

## Create an eval

In [ ]:
eval_obj = client.evals.create(
    ai_description="Minimal SDK Example Eval",
    ai_instructions="Answer the prompts as best as you can.",
    eval_type="safety",
    name="minimal-example-eval",
    num_prompts=5,
)
eval_id = eval_obj.eval_uuid
if not eval_id:
    raise RuntimeError("Eval creation failed.")
eval_obj

## Fetch prompts for the eval

In [ ]:
from aymara_ai.lib.utils import wait_until_complete

eval_obj = wait_until_complete(client.evals.get, resource_id=eval_id)
eval_obj

In [ ]:
from typing import List

from aymara_ai.types.eval_prompt import EvalPrompt

prompts_response = client.evals.list_prompts(eval_id)
prompts: List[EvalPrompt] = prompts_response.items
if not prompts:
    raise RuntimeError("No prompts found for eval.")

## Call OpenAI for each prompt and collect responses

In [ ]:
from typing import List

responses: List[EvalResponseParam] = []
for prompt in prompts:
    prompt_text = prompt.content
    prompt_uuid = prompt.prompt_uuid
    completion = openai.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        prompt=[prompt_text],
        max_tokens=256,
        temperature=0.7,
    )
    answer = completion.choices[0].text.strip()
    responses.append(EvalResponseParam(content=answer, prompt_uuid=prompt_uuid))

## Create an eval run with the responses

In [ ]:
eval_run = client.evals.create_run(eval_uuid=eval_id, responses=responses)
eval_run_id = eval_run.eval_run_uuid
eval_run

In [ ]:
eval_run = wait_until_complete(client.evals.get_run, resource_id=eval_run_id)
eval_run

In [ ]:
report = client.reports.create(eval_run_uuids=[eval_run_id])
report

In [ ]:
report = wait_until_complete(client.reports.get, resource_id=report.eval_suite_report_uuid)
report

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
rows = [s.to_dict() for s in report.eval_run_reports]

df = pd.DataFrame(rows)
df

In [ ]:
from aymara_ai.lib.plot import graph_eval_stats

graph_eval_stats(eval_runs=eval_run)